In [ ]:
!git clone https://github.com/mf1024/ImageNet-Datasets-Downloader.git
!pip install imutils

In [ ]:
import imutils 
import os 
import shutil
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 

In [216]:
import os
import numpy as np
import requests
import argparse
import json
import time
import logging
import csv

from multiprocessing import Pool, Process, Value, Lock
from requests.exceptions import ConnectionError, ReadTimeout, TooManyRedirects, MissingSchema, InvalidURL

debug = False
images_per_class = 1000
use_class_list = True
class_list = ["n02958343",  'n02503517', 'n01323068', "n02129165", 
                                          "n02691156", "n03793489"]
scrape_only_flickr = True
number_of_classes = 10
multiprocessing_workers = 8

IMAGENET_API_WNID_TO_URLS = lambda wnid: f'http://www.image-net.org/api/imagenet.synset.geturls?wnid={wnid}'

current_folder = 'ImageNet-Datasets-Downloader/'

class_info_json_filename = 'imagenet_class_info.json'
class_info_json_filepath = os.path.join(current_folder, class_info_json_filename)

class_info_dict = dict()

with open(class_info_json_filepath) as class_info_json_f:
    class_info_dict = json.load(class_info_json_f)

classes_to_scrape = []

if use_class_list == True:
   for item in class_list:
       classes_to_scrape.append(item)
       if item not in class_info_dict:
           logging.error(f'Class {item} not found in ImageNete')
           exit()

elif use_class_list == False:
    potential_class_pool = []
    for key, val in class_info_dict.items():

        if scrape_only_flickr:
            if int(val['flickr_img_url_count']) * 0.9 > images_per_class:
                potential_class_pool.append(key)
        else:
            if int(val['img_url_count']) * 0.8 > images_per_class:
                potential_class_pool.append(key)

    if (len(potential_class_pool) < number_of_classes):
        logging.error(f"With {images_per_class} images per class there are {len(potential_class_pool)} to choose from.")
        logging.error(f"Decrease number of classes or decrease images per class.")
        exit()

    picked_classes_idxes = np.random.choice(len(potential_class_pool), number_of_classes, replace = False)

    for idx in picked_classes_idxes:
        classes_to_scrape.append(potential_class_pool[idx])


print("Picked the following clases:")
print([ class_info_dict[class_wnid]['class_name'] for class_wnid in classes_to_scrape ])

imagenet_images_folder = os.path.join("simclr_data", 'imagenet_images')
if not os.path.isdir(imagenet_images_folder):
    os.mkdir(imagenet_images_folder)


scraping_stats = dict(
    all=dict(
        tried=0,
        success=0,
        time_spent=0,
    ),
    is_flickr=dict(
        tried=0,
        success=0,
        time_spent=0,
    ),
    not_flickr=dict(
        tried=0,
        success=0,
        time_spent=0,
    )
)

def add_debug_csv_row(row):
    with open('stats.csv', "a") as csv_f:
        csv_writer = csv.writer(csv_f, delimiter=",")
        csv_writer.writerow(row)

class MultiStats():
    def __init__(self):

        self.lock = Lock()

        self.stats = dict(
            all=dict(
                tried=Value('d', 0),
                success=Value('d',0),
                time_spent=Value('d',0),
            ),
            is_flickr=dict(
                tried=Value('d', 0),
                success=Value('d',0),
                time_spent=Value('d',0),
            ),
            not_flickr=dict(
                tried=Value('d', 0),
                success=Value('d', 0),
                time_spent=Value('d', 0),
            )
        )
    def inc(self, cls, stat, val):
        with self.lock:
            self.stats[cls][stat].value += val

    def get(self, cls, stat):
        with self.lock:
            ret = self.stats[cls][stat].value
        return ret

multi_stats = MultiStats()


if False:
    row = [
        "all_tried",
        "all_success",
        "all_time_spent",
        "is_flickr_tried",
        "is_flickr_success",
        "is_flickr_time_spent",
        "not_flickr_tried",
        "not_flickr_success",
        "not_flickr_time_spent"
    ]
    add_debug_csv_row(row)

def add_stats_to_debug_csv():
    row = [
        multi_stats.get('all', 'tried'),
        multi_stats.get('all', 'success'),
        multi_stats.get('all', 'time_spent'),
        multi_stats.get('is_flickr', 'tried'),
        multi_stats.get('is_flickr', 'success'),
        multi_stats.get('is_flickr', 'time_spent'),
        multi_stats.get('not_flickr', 'tried'),
        multi_stats.get('not_flickr', 'success'),
        multi_stats.get('not_flickr', 'time_spent'),
    ]
    add_debug_csv_row(row)

def print_stats(cls, print_func):

    actual_all_time_spent = time.time() - scraping_t_start.value
    processes_all_time_spent = multi_stats.get('all', 'time_spent')

    if processes_all_time_spent == 0:
        actual_processes_ratio = 1.0
    else:
        actual_processes_ratio = actual_all_time_spent / processes_all_time_spent

    #print(f"actual all time: {actual_all_time_spent} proc all time {processes_all_time_spent}")

    print_func(f'STATS For class {cls}:')
    print_func(f' tried {multi_stats.get(cls, "tried")} urls with'
               f' {multi_stats.get(cls, "success")} successes')

    if multi_stats.get(cls, "tried") > 0:
        print_func(f'{100.0 * multi_stats.get(cls, "success")/multi_stats.get(cls, "tried")}% success rate for {cls} urls ')
    if multi_stats.get(cls, "success") > 0:
        print_func(f'{multi_stats.get(cls,"time_spent") * actual_processes_ratio / multi_stats.get(cls,"success")} seconds spent per {cls} succesful image download')



lock = Lock()
url_tries = Value('d', 0)
scraping_t_start = Value('d', time.time())
class_folder = ''
class_images = Value('d', 0)

def get_image(img_url):

    #print(f'Processing {img_url}')

    #time.sleep(3)

    if len(img_url) <= 1:
        return


    cls_imgs = 0
    with lock:
        cls_imgs = class_images.value

    if cls_imgs >= images_per_class:
        return

    logging.debug(img_url)

    cls = ''

    if 'flickr' in img_url:
        cls = 'is_flickr'
    else:
        cls = 'not_flickr'
        if scrape_only_flickr:
            return

    t_start = time.time()

    def finish(status):
        t_spent = time.time() - t_start
        multi_stats.inc(cls, 'time_spent', t_spent)
        multi_stats.inc('all', 'time_spent', t_spent)

        multi_stats.inc(cls,'tried', 1)
        multi_stats.inc('all', 'tried', 1)

        if status == 'success':
            multi_stats.inc(cls,'success', 1)
            multi_stats.inc('all', 'success', 1)

        elif status == 'failure':
            pass
        else:
            logging.error(f'No such status {status}!!')
            exit()
        return


    with lock:
        url_tries.value += 1
        if url_tries.value % 250 == 0:
            print(f'\nScraping stats:')
            print_stats('is_flickr', print)
            print_stats('not_flickr', print)
            print_stats('all', print)
            if debug:
                add_stats_to_debug_csv()

    try:
        img_resp = requests.get(img_url, timeout = 1)
    except ConnectionError:
        logging.debug(f"Connection Error for url {img_url}")
        return finish('failure')
    except ReadTimeout:
        logging.debug(f"Read Timeout for url {img_url}")
        return finish('failure')
    except TooManyRedirects:
        logging.debug(f"Too many redirects {img_url}")
        return finish('failure')
    except MissingSchema:
        return finish('failure')
    except InvalidURL:
        return finish('failure')

    if not 'content-type' in img_resp.headers:
        return finish('failure')

    if not 'image' in img_resp.headers['content-type']:
        logging.debug("Not an image")
        return finish('failure')

    if (len(img_resp.content) < 1000):
        return finish('failure')

    logging.debug(img_resp.headers['content-type'])
    logging.debug(f'image size {len(img_resp.content)}')

    img_name = img_url.split('/')[-1]
    img_name = img_name.split("?")[0]

    if (len(img_name) <= 1):
        return finish('failure')

    img_file_path = os.path.join(class_folder, img_name)
    logging.debug(f'Saving image in {img_file_path}')

    with open(img_file_path, 'wb') as img_f:
        img_f.write(img_resp.content)

        with lock:
            class_images.value += 1

        logging.debug(f'Scraping stats')
        print_stats('is_flickr', logging.debug)
        print_stats('not_flickr', logging.debug)
        print_stats('all', logging.debug)

        return finish('success')


for class_wnid in classes_to_scrape:

    class_name = class_info_dict[class_wnid]["class_name"]
    print(f'Scraping images for class \"{class_name}\"')
    url_urls = IMAGENET_API_WNID_TO_URLS(class_wnid)

    time.sleep(0.05)
    resp = requests.get(url_urls)

    class_folder = os.path.join(imagenet_images_folder, class_name)
    if not os.path.exists(class_folder):
        os.mkdir(class_folder)

    class_images.value = 0

    urls = [url.decode('utf-8') for url in resp.content.splitlines()]

    #for url in  urls:
    #    get_image(url)

    print(f"Multiprocessing workers: {multiprocessing_workers}")
    with Pool(processes=multiprocessing_workers) as p:
        p.map(get_image,urls)


Picked the following clases:
['car', 'elephant', 'tiger cub', 'lion', 'airplane', 'mouse']
Scraping images for class "car"
Multiprocessing workers: 8

Scraping stats:
STATS For class is_flickr:
 tried 242.0 urls with 202.0 successes
83.47107438016529% success rate for is_flickr urls 
0.02288476311334289 seconds spent per is_flickr succesful image download
STATS For class not_flickr:
 tried 0.0 urls with 0.0 successes
STATS For class all:
 tried 242.0 urls with 202.0 successes
83.47107438016529% success rate for all urls 
0.023212544988877704 seconds spent per all succesful image download

Scraping stats:
STATS For class is_flickr:
 tried 492.0 urls with 407.0 successes
82.72357723577235% success rate for is_flickr urls 
0.020271365382428203 seconds spent per is_flickr succesful image download
STATS For class not_flickr:
 tried 0.0 urls with 0.0 successes
STATS For class all:
 tried 494.0 urls with 407.0 successes
82.38866396761134% success rate for all urls 
0.020354603080843237 second

In [217]:
import os 

DATA_DIR = "simclr_data/imagenet_images/"

def rename_files(data_dir):
    """
        this function, will rename the files in the main data directort. the filename will be their category name.
        Params:
            data_dir(type: str): Path, where the simclr imagenet data is saved.
        Return(None)
    """
    for direc in os.listdir(data_dir): 
        try: 
            for indx, file in enumerate(os.listdir(data_dir+direc)): 
                try: 
                    src_filename = data_dir + direc + "/" + file
                    dst_filename = data_dir + direc + "/" + str(direc) + f"_{indx}.jpg"

                    if indx >= 900: 
                        os.remove(src_filename) 

                    if str(direc)[0] != file[0] and indx <= 400: 
                        os.rename(src_filename, dst_filename)
                        
                except Exception as error:
                    continue

        except Exception as error:
            return error 
                

In [ ]:
rename_files(DATA_DIR)

In [267]:
import shutil 
import random
import numpy as np 

def create_train_test_data(data_path, num_train, num_test, num_val): 
    """
        this function, will create a train, test, val directory by random picking the data from simclr imagenet data directory.
        Params:
            data_path(type: str): Path, where the simclr imagenet data is saved, 
                                  so the function can read and put it into new directory.
            num_train(dtype: Int): Number of training examples.
            num_test(dtype: Int): Number of testing examples.
            num_val(dtype: Int): Number of Validation examples.
        Return(None)
    """
    directories = os.listdir(data_path)
    if not os.path.exists("train"):
        os.mkdir("train")
        
    if not os.path.exists("test"):
        os.mkdir("test")
        
    if not os.path.exists("val"):
        os.mkdir("val")
    
    for directory in directories: 
        try: 
            files_list = os.listdir(data_path+directory)
            train_files = np.random.choice(files_list, num_train, replace=False)
            test_files = [file for file in files_list if not file in train_files][: 50]
            val_files = [file for file in files_list if (not file in train_files and not file in test_files)][: 50]
            
            for train_file in train_files:
                src = data_path + directory + '/' + train_file
                dst = "train" + "/" + train_file
                shutil.copyfile(src, dst)

            for test_file in test_files:
                src = data_path + directory + '/' + test_file
                dst = "test" + "/" + test_file
                shutil.copyfile(src, dst)
                
            for val_file in val_files:
                src = data_path + directory + '/' + val_file
                dst = "val" + "/" + val_file
                shutil.copyfile(src, dst)
                
        except Exception as error:
            return error
            

In [ ]:
# creating train test and val directory.
create_train_test_data(data_dir, 340, 50, 50)

In [273]:
# creating a list of paths of train data, test and val data

train_image_files = imutils.paths('train')
test_image_files = imutils.paths('test')
val_image_file = imutils.paths('val')

In [207]:
from tensorflow import keras 
import tensorflow as tf 


# Image preprocessing utils
@tf.function
def parse_images(image_path):
    """
        this function, will read the image file into tensorflow object and resize it using tf.image library.
        Params:
            image_path(type: str): Path, where the images are saved.
        Return(type: tf,Tensor)
            returns the image as tf.Tensor object.
    """
    image_string = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, size=[224, 224])

    return image

def create_tensorflow_dataset(data, batch_size):
    """
        this function, will convert the array of data into tensorflow dataset object.
        Params:
            data(type: np.array): Data, that needed to be converted.
            batch_size(type: Int): Number of batch size.
        Return(type: tf,data.Data)
            returns tensorflow data.Data object
    """
    tensorflow_data = tf.data.Dataset.from_tensor_slices(train_image_files)
    tensorflow_data = (
    tensorflow_data
        .map(parse_images, num_parallel_calls=tf.data.experimental.AUTOTUNE)
        .shuffle(1024)
        .batch(batch_size, drop_remainder=True)
        .prefetch(tf.data.experimental.AUTOTUNE)
    )
    return tensorflow_data        

In [209]:
# convert the images files into tensorflow dataset.

train_ds = create_tensorflow_dataset(train_image_files, 32)
test_ds = create_tensorflow_dataset(test_image_files, 32)
val_ds = create_tensorflow_dataset(test_image_files, 32)